In [1]:
from IPython.display import display
!conda info


     active environment : base
    active env location : /Users/marcelo_palermo/anaconda3
            shell level : 1
       user config file : /Users/marcelo_palermo/.condarc
 populated config files : /Users/marcelo_palermo/.condarc
          conda version : 23.3.1
    conda-build version : 3.24.0
         python version : 3.10.11.final.0
       virtual packages : __archspec=1=x86_64
                          __osx=13.3.1=0
                          __unix=0=0
       base environment : /Users/marcelo_palermo/anaconda3  (writable)
      conda av data dir : /Users/marcelo_palermo/anaconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/conda-forge/osx-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://repo.anaconda.com/pkgs/main/osx-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/osx-64
              

In [2]:
def separate_proteins(filename):
    
    
    training_dataset = pd.read_csv(filename)
    print("Original Data for " + filename)
    display(training_dataset.head(10))
    display(training_dataset.shape)
    #Drop unnecessary columns
    training_dataset = training_dataset.drop(['Unnamed: 0','Threshold','Desc','OlinkID','UniProt','Panel','Organ','Desc_Column','term','df','sumsq','meansq','statistic'], axis=1)
    #training_dataset = training_dataset.drop(['Unnamed: 0','Threshold','OlinkID','UniProt','Panel','term','df','sumsq','meansq','statistic'], axis=1)
    display(training_dataset.head(10))
    display(training_dataset.shape)
    
    
    training_dataframe = pd.DataFrame()
    validation_dataframe = pd.DataFrame()
    
    if(filename.find('train')>0):
        print('This is training file')
        if training_dataframe.empty:
            training_dataframe = training_dataset
            print('creating empty dataframe and placing training_dataset into it')
            
        
        
    else: 
        print('This is validation file')
    
    
    display(training_dataframe.head(10))
    
    return list(training_dataset['Assay'].values)
    

In [3]:
def build_ML_data_prep (filename,df,age):
    print("Processing file " + filename)
    training_dataset1 = pd.read_csv(filename)
    for protein in training_dataset1['Assay']:
        #print(protein)
        if protein in df.columns:
            df.loc[df['AgeCat']==age, [protein]] = 1
        else:
            df.loc[df['AgeCat']==age, [protein]] = 0
    display(df.head())
    return(df)
    
    

In [4]:
import pandas as pd
import numpy as np
import re

In [5]:
from skmultilearn.adapt import MLkNN, MLARAM
# para transformação de problemas usaremos Binary Relevance, Classifier Chain e Label Powerset
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
import pandas as pd
import numpy as np
import sys
#np.set_printoptions(threshold=sys.maxsize)
from sklearn.naive_bayes import GaussianNB,MultinomialNB

### Read MGH Olink ANOVA dataset for the referred category

In [6]:
category = ['age','BMI']
category_range_age = range(1,6)
category_range_BMI = range(0,5)

In [7]:
for i in category_range_age:
    print("Age category:", category_range_age[i-1])
print("\r")
for i in category_range_BMI:
    print("BMI category:", category_range_BMI[i])

Age category: 1
Age category: 2
Age category: 3
Age category: 4
Age category: 5

BMI category: 0
BMI category: 1
BMI category: 2
BMI category: 3
BMI category: 4


In [8]:
import glob, os
files_to_prepare = []
files_to_prepare_train = []

split_factor_file_train = "75_25"
split_factor_file_validation = "75_25"

listing = glob.glob('/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/' + split_factor_file_train + '/MGH_ANOVA_BMI_categories_train_Age_*.csv')

for filename in listing:
    if "cut" in filename:
        print("NOT INCLUDED - Found cut in ", filename)
    else:
        print(filename)
        files_to_prepare.append(filename)
        
## BMI Category 5 training & validation is Unknown --> Discarded
## BMI Category 1 training is empty --> discarded

files_to_prepare.sort()

/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_3.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_4_BMI_4.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_2.csv
NOT INCLUDED - Found cut in  /Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_3_BMI_2_cut.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_1.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_4_BMI_2.csv
NOT INCLUDED - Found cut in  /Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_4_BMI_2_cut.csv
/Users/marcelo_palermo/Doutorado/R/Olink/Next

In [9]:
len(files_to_prepare)

20

In [10]:
files_to_prepare

['/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_1.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_2.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_3.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_4.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_1.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_2.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_3.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/M

### We load validation files just to capture ALL proteins
- Some proteins will appear only in validation
- Some proteins will appear only in train
- We need to ensure ALL columns for Age will be the same in validation and train sets

In [11]:
files_to_prepare_train = files_to_prepare
files_to_prepare_validation = []

listing = glob.glob('/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/' + split_factor_file_validation + '/MGH_ANOVA_BMI_categories_validation_Age_*.csv')
for filename in listing:
    if "cut" in filename:
        print("NOT INCLUDED - Found cut in ", filename)
    else:
        print(filename)
        files_to_prepare_validation.append(filename)
        
        
## BMI Category 5 training & validation is Unknown --> Discarded
## BMI Category 1 training is empty --> discarded

files_to_prepare_validation.sort()

/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_3_BMI_3.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_3_BMI_2.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_3_BMI_1.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_5_BMI_2.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_3_BMI_4.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_5_BMI_3.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_5_BMI_1.csv
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75

In [12]:
files_to_prepare + files_to_prepare_validation

['/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_1.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_2.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_3.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_4.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_1.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_2.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_2_BMI_3.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/M

In [13]:
len(files_to_prepare)

20

In [14]:
files_to_prepare_validation

['/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_1_BMI_4.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_2_BMI_2.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_2_BMI_3.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_2_BMI_4.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_3_BMI_1.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_3_BMI_2.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_3_BMI_3.csv',
 '/Users/marcelo_palermo/Doutorado/R/Olink/NextS

In [15]:
array_protein_names = []
array_each_file = []
for file in (files_to_prepare + files_to_prepare_validation):
    array_each_file = separate_proteins(file)
    array_protein_names = array_protein_names + array_each_file

Original Data for /Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_1.csv


,Unnamed: 0,Assay,OlinkID,UniProt,Panel,term,df,sumsq,meansq,statistic,p.value,Adjusted_pval,Threshold,Organ,Desc,Desc_Column


(0, 16)

,Assay,p.value,Adjusted_pval


(0, 3)

...

(0, 16)

,Assay,p.value,Adjusted_pval


(0, 3)

This is validation file


""


### Protein Atlas

https://www.proteinatlas.org/humanproteome/tissue/tissue+specific

### Scientific reference
- https://onlinelibrary.wiley.com/doi/abs/10.1002/pro.3987 
- https://doi.org/10.1002/pro.3987 

### Remove all duplicated protein names for age categories 1-5 in ```files_to_prepare```

In [16]:
array_protein_names

['REG3A',
 'EREG',
 'HAVCR1',
 'NEFL',
 'TGFBR2',
 'TNFRSF4',
 'EFNA4',
 'FAS',
 'GFOD2',
 'IGFBP4',
 'MMP7',
 'RELT',
 'TNFRSF9',
 'IFNGR1',
 'PIK3IP1',
 'SDC1',
 'TNFRSF11A',
 'IL1B',
 'CD27',
 'CD302',
 'APOH',
 'BAG3',
 'EDA2R',
 'FCER2',
 'PTN',
 'PTPRF',
 'RGMB',
 'TNFRSF14',
 'ULBP2',
 'DDC',
 'DEFA1_DEFA1B',
 'LAYN',
 'RARRES2',
 'TFPI2',
 'TNFRSF19',
 'WFDC2',
 'ACE2',
 'S100A11',
 'NPPB',
 'DPP10',
 'PAMR1',
 'CD5',
 'JAM2',
 'SPINK4',
 'TNFRSF12A',
 'RSPO3',
 'DSC2',
 'CCL28',
 'ART3',
 'COL18A1',
 'HYOU1',
 'NECTIN2',
 'PDCD1',
 'FAM3C',
 'ADGRB3',
 'CD79B',
 'CRX',
 'CTSF',
 'FGFBP1',
 'HSPG2',
 'IGF1R',
 'IL10RB',
 'NBL1',
 'PIGR',
 'TNFRSF1A',
 'VTCN1',
 'SFRP1',
 'SPINT1',
 'GPC1',
 'SPINK1',
 'COLEC12',
 'NTproBNP',
 'FSTL3',
 'SCARF2',
 'SLAMF6',
 'BAIAP2',
 'CD300LG',
 'CD99',
 'NCS1',
 'SIT1',
 'ARTN',
 'CD160',
 'TNFRSF10C',
 'CD93',
 'EFNA1',
 'LRP11',
 'RBP2',
 'RBP5',
 'S100P',
 'ANXA3',
 'ADAM22',
 'MDK',
 'LY96',
 'RSPO1',
 'GDF15',
 'BGN',
 'BTN2A1',
 'CASP1'

In [17]:
#Size with duplicated proteins
len(array_protein_names)

1656

In [18]:
#Test how to find a specific protein in list
list(array_protein_names)[0]

'REG3A'

In [19]:
#Compare array_protein_names size after removing duplicates
len(set(array_protein_names))

847

In [20]:
# Achar um elemento na lista - Testar se há duplicados
for i in array_protein_names:
    if (i=='SDC1'):
        print (i)
        

SDC1
SDC1
SDC1
SDC1
SDC1
SDC1
SDC1


### Create a pandas dataframe containing unique protein names as columns from <i>set(array_protein_names)</i>

In [21]:
df = pd.DataFrame(columns = list(set(array_protein_names)))
df["Age"] = ""
df["BMI"] = ""

In [22]:
df.head()

,GP1BA,ADAM8,LAIR2,PDGFRA,MMP12,BAX,CNTN1,EDIL3,MNDA,STX4,...,DNMBP,APLP1,ITGA5,SLIT2,SELPLG,DLK1,GPC1,WASF1,Age,BMI


In [23]:
#Confirming list size for unique protein names
len(list(set(array_protein_names)))

847

In [24]:
#Test - finding the column index of a certain protein name
df.columns.get_loc("SCARB2") ##localizacao da coluna no dataframe

328

In [25]:
#Test whether a protein exists in df columns
if 'SCARB2' in df.columns:
    print("exists")
    print(df.columns.get_loc("SCARB2"))
else:
    print('SCARB2 Does not Exist')

exists
328


## Build Train dataframe

In [26]:
df_validation = df

In [27]:
training_dataframe = pd.DataFrame()
for file in files_to_prepare_train:
    print(file)
    train = pd.read_csv(file)
    #train = train.drop(['Unnamed: 0','Threshold','OlinkID','p.value','UniProt','Adjusted_pval','Panel','term','df','sumsq','meansq','statistic'], axis=1)
    train = train.drop(['Unnamed: 0','Threshold','OlinkID','p.value','UniProt','Adjusted_pval','Panel','term','df','sumsq','meansq','statistic', 'Organ', 'Desc','Desc_Column'], axis=1)


    display(train.head(10))
    if (train.shape[0]==0):
        print("Vazio")
        #Não adicionar Colunas Age e BMI
    else:
        print("NÃO VAZIO")
        print("Age: " + re.findall(r'\d+', file)[2])
        print("BMI: " + re.findall(r'\d+', file)[3])
        train['AgeCat'] = re.findall(r'\d+', file)[2]
        train['BMI'] = re.findall(r'\d+', file)[3]
        display(train.head(10))
        
        
        train_label_encoded = pd.get_dummies(train[['Assay']], prefix=None)
        train_label_encoded.columns = [col.replace('Assay_', '') for col in train_label_encoded.columns]
        display(train_label_encoded)
        
        train_label_encoded['Age']=re.findall(r'\d+', file)[2]
        train_label_encoded['BMI']=re.findall(r'\d+', file)[3]
        
        
        #Test whether a protein exists in df columns
        if 'Age' in train_label_encoded.columns:
            print("exists --->" + str(train_label_encoded.columns.get_loc("Age")))
        else:
            print('Does not Exist')
            
            
        #partial save CSV
        train_label_encoded.to_csv('/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/Prepared_Data_for_ML_Input/PreparedData_train_' + split_factor_file_train + '_Age_'+re.findall(r'\d+', file)[2]+'_BMI_'+re.findall(r'\d+', file)[3] + '.csv')
        
        display(train_label_encoded)
        display(df)
        
        df = df.merge(train_label_encoded, how='outer')
        df.fillna(0, inplace=True)
        
        
        
            
        
        
        #df['rating'] = np.where(df['line_race']==0, 0, df['rating'])
        #for i, j in train.iterrows():
        #    print(i, j)
df = df.astype(int)
display(df)     
        
        

/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_1.csv


,Assay


Vazio
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_2.csv


,Assay


Vazio
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_train_Age_1_BMI_3.csv


...

,ADGRG2,CCL18,CDSN,CLC,CST5,LY96,PADI2,TIMP3,TNNI3
0,False,False,False,False,False,False,True,False,False
1,False,False,False,True,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False
5,False,False,False,False,False,True,False,False,False
6,False,False,False,False,False,False,False,True,False
7,False,False,False,False,False,False,False,False,True
8,False,False,False,False,True,False,False,False,False


exists --->9


,ADGRG2,CCL18,CDSN,CLC,CST5,LY96,PADI2,TIMP3,TNNI3,Age,BMI
0,False,False,False,False,False,False,True,False,False,5,3
1,False,False,False,True,False,False,False,False,False,5,3
2,True,False,False,False,False,False,False,False,False,5,3
3,False,True,False,False,False,False,False,False,False,5,3
4,False,False,True,False,False,False,False,False,False,5,3
5,False,False,False,False,False,True,False,False,False,5,3
6,False,False,False,False,False,False,False,True,False,5,3
7,False,False,False,False,False,False,False,False,True,5,3
8,False,False,False,False,True,False,False,False,False,5,3


,GP1BA,ADAM8,LAIR2,PDGFRA,MMP12,BAX,CNTN1,EDIL3,MNDA,STX4,...,APLP1,ITGA5,SLIT2,SELPLG,DLK1,GPC1,WASF1,REG3A,Age,BMI
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,True,1,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,False,0.0,0,1,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,False,0.0,0,1,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,False,0.0,0,1,4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,False,0.0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,False,False,0,False,False,False,False,False,False,False,...,0,False,False,False,False,False,False,False,4,2
1638,False,False,0,False,False,False,False,False,False,False,...,0,False,False,False,False,False,False,False,4,2
1639,False,False,0,False,False,False,False,False,False,False,...,0,False,False,False,False,False,False,False,4,2
1640,False,False,0,False,False,False,False,False,False,False,...,0,False,False,False,False,False,False,False,4,2


,GP1BA,ADAM8,LAIR2,PDGFRA,MMP12,BAX,CNTN1,EDIL3,MNDA,STX4,...,APLP1,ITGA5,SLIT2,SELPLG,DLK1,GPC1,WASF1,REG3A,Age,BMI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3
1647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3
1648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3
1649,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3


In [28]:
df.head()

,GP1BA,ADAM8,LAIR2,PDGFRA,MMP12,BAX,CNTN1,EDIL3,MNDA,STX4,...,APLP1,ITGA5,SLIT2,SELPLG,DLK1,GPC1,WASF1,REG3A,Age,BMI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4


### Replace NaN with Zeroes
This will serve as one hot encoding

In [29]:
#df = df.replace(np.nan,0)


### Walk through df columns and fulfill 1 for existent proteins in each age_train file
This routine will go through Age train files 1 to 5

In [30]:
columns = list(df.columns)

In [31]:
type(columns)

list

In [32]:
columns[0]

'GP1BA'

In [33]:
len(df), df.shape

(1651, (1651, 849))

In [34]:
#df = df.groupby(['Age','BMI']).count()
#print("Sum")
#display(df)

In [35]:
df = df.reset_index()
df.head()

,index,GP1BA,ADAM8,LAIR2,PDGFRA,MMP12,BAX,CNTN1,EDIL3,MNDA,...,APLP1,ITGA5,SLIT2,SELPLG,DLK1,GPC1,WASF1,REG3A,Age,BMI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,3
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4


In [36]:
df.to_csv('/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/Prepared_Data_for_ML_Input/PreparedData_train_Age_BMI_' + split_factor_file_train + '.csv')

## Build validation dataframe

In [37]:
training_dataframe = pd.DataFrame()
for file in files_to_prepare_validation:
    print(file)
    validation = pd.read_csv(file)
    #validation = validation.drop(['Unnamed: 0','Threshold','OlinkID','p.value','UniProt','Adjusted_pval','Panel','term','df','sumsq','meansq','statistic'], axis=1)
    validation = validation.drop(['Unnamed: 0','Threshold','OlinkID','p.value','UniProt','Adjusted_pval','Panel','term','df','sumsq','meansq','statistic', 'Organ', 'Desc','Desc_Column'], axis=1)

    
    
    display(validation.head(10))
    if (validation.shape[0]==0):
        print("Vazio")
        #Não adicionar Colunas Age e BMI
    else:
        print("NÃO VAZIO")
        print("Age: " + re.findall(r'\d+', file)[2])
        print("BMI: " + re.findall(r'\d+', file)[3])
        validation['AgeCat'] = re.findall(r'\d+', file)[2]
        validation['BMI'] = re.findall(r'\d+', file)[3]
        display(validation.head(10))
        
        
        validation_label_encoded = pd.get_dummies(validation[['Assay']], prefix=None)
        validation_label_encoded.columns = [col.replace('Assay_', '') for col in validation_label_encoded.columns]
        display(validation_label_encoded)
        
        validation_label_encoded['Age']=re.findall(r'\d+', file)[2]
        validation_label_encoded['BMI']=re.findall(r'\d+', file)[3]
        
        
        #Test whether a protein exists in df columns
        if 'Age' in validation_label_encoded.columns:
            print("exists --->" + str(validation_label_encoded.columns.get_loc("Age")))
        else:
            print('Does not Exist')
            
            
        #partial save CSV
        validation_label_encoded.to_csv('/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/Prepared_Data_for_ML_Input/PreparedData_validation_' + split_factor_file_validation + '_Age_'+re.findall(r'\d+', file)[2]+'_BMI_'+re.findall(r'\d+', file)[3] + '.csv')
        
        display(validation_label_encoded)
        display(df_validation)
        
        df_validation = df_validation.merge(validation_label_encoded, how='outer')
        df_validation.fillna(0, inplace=True)
        
        #df é a hot-encoded
        #atualizar cada linha com 1 onde a coluna é igual a row["Assay"'
        
            
        
        
        #df['rating'] = np.where(df['line_race']==0, 0, df['rating'])
        #for i, j in train.iterrows():
        #    print(i, j)
df_validation = df_validation.astype(int)
display(df_validation)     

/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_1_BMI_4.csv


,Assay


Vazio
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_2_BMI_2.csv


,Assay


Vazio
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_2_BMI_3.csv


...

,Assay


Vazio
/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/AGE_BMI_Significatives/75_25/MGH_ANOVA_BMI_categories_validation_Age_5_BMI_3.csv


,Assay


Vazio


,GP1BA,ADAM8,LAIR2,PDGFRA,MMP12,BAX,CNTN1,EDIL3,MNDA,STX4,...,APLP1,ITGA5,SLIT2,SELPLG,DLK1,GPC1,WASF1,VTCN1,Age,BMI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,1


In [38]:
df_validation.head()

,GP1BA,ADAM8,LAIR2,PDGFRA,MMP12,BAX,CNTN1,EDIL3,MNDA,STX4,...,APLP1,ITGA5,SLIT2,SELPLG,DLK1,GPC1,WASF1,VTCN1,Age,BMI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,1


In [39]:
columns_validation = list(df_validation.columns)

In [40]:
type(columns_validation)

list

In [41]:
columns_validation[0]

'GP1BA'

In [42]:
len(df_validation), df_validation.shape

(5, (5, 849))

In [43]:
df_validation.to_csv('/Users/marcelo_palermo/Doutorado/R/Olink/NextSteps/Prepared_Data_for_ML_Input/PreparedData_validation_Age_BMI_' + split_factor_file_validation + '.csv')

In [44]:
#df_validation = df_validation.groupby(['Age','BMI']).count()
#print("Sum")
#display(df_validation)

In [45]:
#df_validation = df_validation.reset_index()
#df_validation.head()

In [46]:
exit()